# Libraries

In [1]:
import pandas as pd 
import json
import random 
import time
import hashlib


from kafka import KafkaProducer
from confluent_kafka import Producer
import socket
import json
import os

from kafka_auth import conf
from utils import get_pay_method,get_coords,create_masive_users



# Purchase Simulation 

In [2]:
df = pd.read_excel('ecomerce_datexland.xlsx')
df

,PRODUCT_NAME,PRICING,COMISION,BRAND,CATEGORY
0,Laptop_Brand1_4GB,1350000.0,0.20,Brand1,LAPTOPS
1,Laptop_Brand1_8GB,2550000.0,0.25,Brand1,LAPTOPS
2,Laptop_Brand1_16GB,3500000.0,0.30,Brand1,LAPTOPS
3,Laptop_Brand1_32GB,4800000.0,0.35,Brand1,GAMING
4,Laptop_Brand2_4GB,1150000.0,0.15,Brand2,LAPTOPS
5,Laptop_Brand2_8GB,1850000.0,0.18,Brand2,LAPTOPS
6,Laptop_Brand2_16GB,3890000.0,0.20,Brand2,LAPTOPS
7,Laptop_Brand2_32GB,6990000.0,0.25,Brand2,GAMING
8,Laptop_Brand3_4GB,1850000.0,0.20,Brand3,LAPTOPS
9,Laptop_Brand3_8GB,3199000.0,0.28,Brand3,LAPTOPS


In [3]:
cities = ['Bogotá','Medellín','Cali','Bucaramanga','Barranquilla']
payment_online = ['Credit_card','PSE']
payment_store = ['Cash','Nequi','Daviplata','Credit_card']
source = ['Facebook','Instagram','Organic','Twitter','Influencer_1','Influencer_2','Influencer_3','Influencer_4']
status_purchase = ['COMPLETED','FAILED_CHECKOUT','FAILED_API_RESPONSE','INSUFICCIENT_FUNDS','COMPLETED','COMPLETED','COMPLETED','COMPLETED','COMPLETED','COMPLETED','FAILED_API_RESPONSE','INSUFICCIENT_FUNDS','USER_ERROR','FRAUD','COMPLETED','COMPLETED','COMPLETED']
#status_purchase = ['FAILED_CHECKOUT','FAILED_API_RESPONSE','INSUFICCIENT_FUNDS','FAILED_API_RESPONSE','FAILED_CHECKOUT','FAILED_API_RESPONSE','INSUFICCIENT_FUNDS','FAILED_API_RESPONSE','INSUFICCIENT_FUNDS','USER_ERROR','FRAUD','COMPLETED']

In [4]:
pay = get_pay_method('Organic',status_purchase,payment_online,payment_store)
print(pay)

('Nequi', 'COMPLETED', 'STORE')


In [5]:
city_test = random.choice(cities)
print(f' Ciudad  >> {city_test}')
print(f' Coordenadas >> {get_coords(city_test)}')

 Ciudad  >> Cali
 Coordenadas >> (3.4148803158142966, -76.54041613631288)


In [6]:
users = create_masive_users(1000)
users

['1b393694ab',
 '700d7ccad5',
 '10e37cfcaf',
 '58e0ff2f18',
 'afb9f5a9ee',
 '21cee42a3e',
 '31c47ad19a',
 '42745241a8',
 '246d8feb95',
 'e8a32df21b',
 '16d0dce1d1',
 '6925ec4671',
 '4309e61852',
 'a194ca0c3d',
 '4340c750d2',
 '0f28c174c5',
 'a5cce3b649',
 'b7d68bfbfd',
 'af2d867b3a',
 'c30e7ad544',
 '25e34c0b8e',
 '070684ec6a',
 '49ffa2ef78',
 'c8c11291be',
 'f3672e5bcc',
 '297255cdde',
 '037e8d5ab7',
 'fd0bfc3468',
 '17df4cfbd2',
 '804f476af1',
 '9d3cd5055d',
 'a8738810ac',
 '63ecfcaf97',
 '035481a20f',
 'f8acbc36b9',
 'ea231708d9',
 'cc2e4d117d',
 '7ec290ae9a',
 '6475a85ba1',
 '71e5f53fa5',
 'e674cd7037',
 '914fa5be44',
 '6307c84e77',
 'c317bc5c35',
 'e21174aedc',
 '494cf5f7c5',
 '59f913527a',
 '08a8537376',
 'f57416ca5c',
 '34ed4428fb',
 'bc6e600403',
 'ab04dc84ef',
 'e8beed262c',
 '5c5152f7a5',
 '4433f713f7',
 '27ca6dbfc8',
 'bc8d9ae616',
 '29784ff4da',
 'ebe3f1966c',
 'c43fde2a94',
 '3dbaa12367',
 'cc6960776f',
 '4ed7d8e2d2',
 '02e93dcbd1',
 '63bc559fbe',
 '37ba3c5708',
 '5bd64728

# To Do's

*  Deploy a kafka Cluster on Confluent Cloud 
* Create a Kafka topic
* Test Python simulation Script

# Config Kafka Cluster

In [7]:
producer = Producer(conf)

In [8]:
#conf

# Send Message To Kafka Topic

In [9]:
delivered_records = 0
topic = 'topic_purchase' 

x = 1

data_purchase = []
#messages = []

while(x > 0):
    
    date = pd.to_datetime('today').strftime("%Y-%m-%d %H:%M:%S")
    
    product = df['PRODUCT_NAME '][random.randint(0,14)]
    pricing =  df[df['PRODUCT_NAME ']==product]['PRICING'].values[0]
    comision =  df[df['PRODUCT_NAME ']==product]['COMISION'].values[0]
    brand = df[df['PRODUCT_NAME ']==product]['BRAND'].values[0]
    category = df[df['PRODUCT_NAME ']==product]['CATEGORY'].values[0]
    source_temp = random.choice(source)
    pay = get_pay_method(source_temp,status_purchase,payment_online,payment_store)
    city = random.choice(cities)
    
    purchase = {'purchase_ID': str(hashlib.sha256(f"{x} {product} {pricing} {comision} {date} {source_temp} {pay[1]}".encode('utf-8')).hexdigest())[:10], 
            'user_id' : random.choices(users)[0] ,
            'Product_name' : product,
            'Pricing':str(pricing),
            'Comision':str(comision),
            'Revenue' : str(round(pricing*comision,2)),
            'Payment_Mehtod':pay[0],
            'Status' : pay[1],
            'Order_Type' : pay[2],
            'City':city,
            'Location': str(get_coords(city)), #[get_coords(city)],
            'Latitud' : str(get_coords(city)[0]) ,
            'Longitud' :  str(get_coords(city)[1]),
            'Source':source_temp,
            'Brand' : brand,
            'Category': category,
            'Created_at': date}
    
    print(purchase)
    data_purchase.append(pd.DataFrame(purchase,index = [x]))
    #messages.append(purchase)

    record_key = "Purchase_simulator"
    record_value =  json.dumps(purchase).encode('utf-8')
    print("Producing record: {}\t{}".format(record_key, record_value))
    producer.produce(topic, key=record_key, value=record_value)
    producer.poll(0)

    delivered_records += 1
    x += 1
    
    time.sleep(random.choice([1,1.5]))

    
    

producer.flush()

print('\n')
print('\n')
print("{} messages were produced to topic {}!".format(delivered_records, topic))
print('\n')
print('\n')

{'purchase_ID': '8657bbec12', 'user_id': '8d43bf26f1', 'Product_name': 'Laptop_Brand1_4GB', 'Pricing': '1350000.0', 'Comision': '0.2', 'Revenue': '270000.0', 'Payment_Mehtod': 'Nequi', 'Status': 'COMPLETED', 'Order_Type': 'STORE', 'City': 'Cali', 'Location': '(3.4287786094866717, -76.53749228193497)', 'Latitud': '3.4164091379805432', 'Longitud': '-76.54751692551635', 'Source': 'Organic', 'Brand': 'Brand1', 'Category': 'LAPTOPS', 'Created_at': '2023-04-15 18:21:09'}
Producing record: Purchase_simulator	b'{"purchase_ID": "8657bbec12", "user_id": "8d43bf26f1", "Product_name": "Laptop_Brand1_4GB", "Pricing": "1350000.0", "Comision": "0.2", "Revenue": "270000.0", "Payment_Mehtod": "Nequi", "Status": "COMPLETED", "Order_Type": "STORE", "City": "Cali", "Location": "(3.4287786094866717, -76.53749228193497)", "Latitud": "3.4164091379805432", "Longitud": "-76.54751692551635", "Source": "Organic", "Brand": "Brand1", "Category": "LAPTOPS", "Created_at": "2023-04-15 18:21:09"}'
{'purchase_ID': '736

{'purchase_ID': 'd5b1f98432', 'user_id': 'fea788f519', 'Product_name': 'Laptop_Brand3_4GB', 'Pricing': '1850000.0', 'Comision': '0.2', 'Revenue': '370000.0', 'Payment_Mehtod': 'PSE', 'Status': 'COMPLETED', 'Order_Type': 'ONLINE', 'City': 'Bogotá', 'Location': '(4.647569693587232, -74.10186525959672)', 'Latitud': '4.651600960108946', 'Longitud': '-74.12628850377475', 'Source': 'Influencer_1', 'Brand': 'Brand3', 'Category': 'LAPTOPS', 'Created_at': '2023-04-15 18:21:19'}
Producing record: Purchase_simulator	b'{"purchase_ID": "d5b1f98432", "user_id": "fea788f519", "Product_name": "Laptop_Brand3_4GB", "Pricing": "1850000.0", "Comision": "0.2", "Revenue": "370000.0", "Payment_Mehtod": "PSE", "Status": "COMPLETED", "Order_Type": "ONLINE", "City": "Bogot\\u00e1", "Location": "(4.647569693587232, -74.10186525959672)", "Latitud": "4.651600960108946", "Longitud": "-74.12628850377475", "Source": "Influencer_1", "Brand": "Brand3", "Category": "LAPTOPS", "Created_at": "2023-04-15 18:21:19"}'
{'purc

{'purchase_ID': 'f768ebda3a', 'user_id': 'be6e30052c', 'Product_name': 'Console_Switch', 'Pricing': '2249000.0', 'Comision': '0.2', 'Revenue': '449800.0', 'Payment_Mehtod': 'Credit_card', 'Status': 'COMPLETED', 'Order_Type': 'ONLINE', 'City': 'Medellín', 'Location': '(6.198053256721469, -75.5733524126965)', 'Latitud': '6.198053256721469', 'Longitud': '-75.59141059178306', 'Source': 'Influencer_1', 'Brand': 'Nintendo', 'Category': 'VIDEO GAMES', 'Created_at': '2023-04-15 18:21:30'}
Producing record: Purchase_simulator	b'{"purchase_ID": "f768ebda3a", "user_id": "be6e30052c", "Product_name": "Console_Switch", "Pricing": "2249000.0", "Comision": "0.2", "Revenue": "449800.0", "Payment_Mehtod": "Credit_card", "Status": "COMPLETED", "Order_Type": "ONLINE", "City": "Medell\\u00edn", "Location": "(6.198053256721469, -75.5733524126965)", "Latitud": "6.198053256721469", "Longitud": "-75.59141059178306", "Source": "Influencer_1", "Brand": "Nintendo", "Category": "VIDEO GAMES", "Created_at": "2023-

{'purchase_ID': 'd37d67924b', 'user_id': 'e68efb8ed4', 'Product_name': 'Laptop_Brand1_16GB', 'Pricing': '3500000.0', 'Comision': '0.3', 'Revenue': '1050000.0', 'Payment_Mehtod': 'Credit_card', 'Status': 'FRAUD', 'Order_Type': 'ONLINE', 'City': 'Bogotá', 'Location': '(4.661984763443271, -74.13466548843223)', 'Latitud': '4.647569693587232', 'Longitud': '-74.10186525959672', 'Source': 'Twitter', 'Brand': 'Brand1', 'Category': 'LAPTOPS', 'Created_at': '2023-04-15 18:21:40'}
Producing record: Purchase_simulator	b'{"purchase_ID": "d37d67924b", "user_id": "e68efb8ed4", "Product_name": "Laptop_Brand1_16GB", "Pricing": "3500000.0", "Comision": "0.3", "Revenue": "1050000.0", "Payment_Mehtod": "Credit_card", "Status": "FRAUD", "Order_Type": "ONLINE", "City": "Bogot\\u00e1", "Location": "(4.661984763443271, -74.13466548843223)", "Latitud": "4.647569693587232", "Longitud": "-74.10186525959672", "Source": "Twitter", "Brand": "Brand1", "Category": "LAPTOPS", "Created_at": "2023-04-15 18:21:40"}'
{'pu

{'purchase_ID': '6697f23329', 'user_id': 'e46b4a96a9', 'Product_name': 'Laptop_Brand2_4GB', 'Pricing': '1150000.0', 'Comision': '0.15', 'Revenue': '172500.0', 'Payment_Mehtod': 'Cash', 'Status': 'COMPLETED', 'Order_Type': 'STORE', 'City': 'Cali', 'Location': '(3.4148803158142966, -76.54041613631288)', 'Latitud': '3.4148803158142966', 'Longitud': '-76.54041613631288', 'Source': 'Organic', 'Brand': 'Brand2', 'Category': 'LAPTOPS', 'Created_at': '2023-04-15 18:21:51'}
Producing record: Purchase_simulator	b'{"purchase_ID": "6697f23329", "user_id": "e46b4a96a9", "Product_name": "Laptop_Brand2_4GB", "Pricing": "1150000.0", "Comision": "0.15", "Revenue": "172500.0", "Payment_Mehtod": "Cash", "Status": "COMPLETED", "Order_Type": "STORE", "City": "Cali", "Location": "(3.4148803158142966, -76.54041613631288)", "Latitud": "3.4148803158142966", "Longitud": "-76.54041613631288", "Source": "Organic", "Brand": "Brand2", "Category": "LAPTOPS", "Created_at": "2023-04-15 18:21:51"}'
{'purchase_ID': '479

...

{'purchase_ID': '418d68ce71', 'user_id': '4963ba777f', 'Product_name': 'Laptop_Brand2_8GB', 'Pricing': '1850000.0', 'Comision': '0.18', 'Revenue': '333000.0', 'Payment_Mehtod': 'Credit_card', 'Status': 'COMPLETED', 'Order_Type': 'ONLINE', 'City': 'Bucaramanga', 'Location': '(7.099917472863198, -73.10730617492302)', 'Latitud': '7.099917472863198', 'Longitud': '-73.10730617492302', 'Source': 'Twitter', 'Brand': 'Brand2', 'Category': 'LAPTOPS', 'Created_at': '2023-04-15 19:13:52'}
Producing record: Purchase_simulator	b'{"purchase_ID": "418d68ce71", "user_id": "4963ba777f", "Product_name": "Laptop_Brand2_8GB", "Pricing": "1850000.0", "Comision": "0.18", "Revenue": "333000.0", "Payment_Mehtod": "Credit_card", "Status": "COMPLETED", "Order_Type": "ONLINE", "City": "Bucaramanga", "Location": "(7.099917472863198, -73.10730617492302)", "Latitud": "7.099917472863198", "Longitud": "-73.10730617492302", "Source": "Twitter", "Brand": "Brand2", "Category": "LAPTOPS", "Created_at": "2023-04-15 19:13:

{'purchase_ID': '09db0b69ed', 'user_id': '7fb3cafcc0', 'Product_name': 'Laptop_Brand3_32GB', 'Pricing': '18999000.0', 'Comision': '0.4', 'Revenue': '7599600.0', 'Payment_Mehtod': 'PSE', 'Status': 'COMPLETED', 'Order_Type': 'ONLINE', 'City': 'Bucaramanga', 'Location': '(7.07243688331635, -73.10525936227518)', 'Latitud': '7.07243688331635', 'Longitud': '-73.10525936227518', 'Source': 'Instagram', 'Brand': 'Brand3', 'Category': 'GAMING', 'Created_at': '2023-04-15 19:14:03'}
Producing record: Purchase_simulator	b'{"purchase_ID": "09db0b69ed", "user_id": "7fb3cafcc0", "Product_name": "Laptop_Brand3_32GB", "Pricing": "18999000.0", "Comision": "0.4", "Revenue": "7599600.0", "Payment_Mehtod": "PSE", "Status": "COMPLETED", "Order_Type": "ONLINE", "City": "Bucaramanga", "Location": "(7.07243688331635, -73.10525936227518)", "Latitud": "7.07243688331635", "Longitud": "-73.10525936227518", "Source": "Instagram", "Brand": "Brand3", "Category": "GAMING", "Created_at": "2023-04-15 19:14:03"}'
{'purcha

{'purchase_ID': '4e8e41fb5d', 'user_id': '59dfb2bf60', 'Product_name': 'Laptop_Brand1_8GB', 'Pricing': '2550000.0', 'Comision': '0.25', 'Revenue': '637500.0', 'Payment_Mehtod': 'PSE', 'Status': 'INSUFICCIENT_FUNDS', 'Order_Type': 'ONLINE', 'City': 'Cali', 'Location': '(3.4287786094866717, -76.53749228193497)', 'Latitud': '3.4287786094866717', 'Longitud': '-76.54751692551635', 'Source': 'Facebook', 'Brand': 'Brand1', 'Category': 'LAPTOPS', 'Created_at': '2023-04-15 19:14:14'}
Producing record: Purchase_simulator	b'{"purchase_ID": "4e8e41fb5d", "user_id": "59dfb2bf60", "Product_name": "Laptop_Brand1_8GB", "Pricing": "2550000.0", "Comision": "0.25", "Revenue": "637500.0", "Payment_Mehtod": "PSE", "Status": "INSUFICCIENT_FUNDS", "Order_Type": "ONLINE", "City": "Cali", "Location": "(3.4287786094866717, -76.53749228193497)", "Latitud": "3.4287786094866717", "Longitud": "-76.54751692551635", "Source": "Facebook", "Brand": "Brand1", "Category": "LAPTOPS", "Created_at": "2023-04-15 19:14:14"}'


{'purchase_ID': 'fd31cfb123', 'user_id': '02fc364063', 'Product_name': 'Console_Xbox', 'Pricing': '1599000.0', 'Comision': '0.15', 'Revenue': '239850.0', 'Payment_Mehtod': 'Credit_card', 'Status': 'COMPLETED', 'Order_Type': 'ONLINE', 'City': 'Barranquilla', 'Location': '(11.004041676077495, -74.83545204769058)', 'Latitud': '11.014167139030025', 'Longitud': '-74.78876005521157', 'Source': 'Facebook', 'Brand': 'X_Box', 'Category': 'VIDEO GAMES', 'Created_at': '2023-04-15 19:14:25'}
Producing record: Purchase_simulator	b'{"purchase_ID": "fd31cfb123", "user_id": "02fc364063", "Product_name": "Console_Xbox", "Pricing": "1599000.0", "Comision": "0.15", "Revenue": "239850.0", "Payment_Mehtod": "Credit_card", "Status": "COMPLETED", "Order_Type": "ONLINE", "City": "Barranquilla", "Location": "(11.004041676077495, -74.83545204769058)", "Latitud": "11.014167139030025", "Longitud": "-74.78876005521157", "Source": "Facebook", "Brand": "X_Box", "Category": "VIDEO GAMES", "Created_at": "2023-04-15 19

KeyboardInterrupt: 

In [ ]:
print('\n')
print('\n')
print("{} messages were produced to topic {}!".format(delivered_records, topic))
print('\n')
print('\n')

In [ ]:
mensaje = {"purchase_ID": "c4cfe85bb4", "user_id": "e0b084c7df", "Product_name": "Laptop_Brand2_8GB", "Pricing": "1850000.0", "Comision": "0.18", "Revenue": "333000.0", "Payment_Mehtod": "Credit_card", "Status": "FAILED_API_RESPONSE", "Order_Type": "ONLINE", "City": "Barranquilla", "Location": "(10.990580146449894, -74.78876005521157)", "Latitud": "11.004041676077495", "Longitud": "-74.83545204769058", "Source": "Influencer_2", "Brand": "Brand2", "Category": "LAPTOPS", "Created_at": "2023-04-15 13:51:38"}

In [ ]:
enrich_purchase = []

for purchase_records in messages : 
    
    transform_records = {'purchase_ID': purchase_records['purchase_ID'], 
        'user_id' : purchase_records['user_id'] ,
        'Product_name' : purchase_records['Product_name'],
        'Pricing':purchase_records['Pricing'],
        'Comision':purchase_records['Comision'],
        'Revenue' : purchase_records['Revenue'],
        'Payment_Mehtod':purchase_records['Payment_Mehtod'],
        'Status' : purchase_records['Status'],
        'Order_Type' : purchase_records['Order_Type'],
        'City':purchase_records['City'],
        'Location': purchase_records['Location'],
        'Latitud' : purchase_records['Latitud'],
        'Longitud' :  purchase_records['Longitud'],
        'Source':purchase_records['Source'],
        'Brand' : purchase_records['Brand'],
        'Category': purchase_records['Category'],
        'Created_at': purchase_records['Created_at'],
        'Date' : purchase_records['Created_at'].split()[0],
        'Hour' : int(purchase_records['Created_at'].split()[1].split(':')[0])
           }
    
    enrich_purchase.append(transform_records)

In [ ]:
enrich_purchase

In [ ]:
purchase = {'purchase_ID': mensaje['purchase_ID'], 
        'user_id' : mensaje['user_id'] ,
        'Product_name' : mensaje['purProduct_namechase_ID'],
        'Pricing':mensaje['Pricing'],
        'Comision':mensaje['Comision'],
        'Revenue' : mensaje['Revenue'],
        'Payment_Mehtod':mensaje['Payment_Mehtod'],
        'Status' : mensaje['Status'],
        'Order_Type' : mensaje['Order_Type'],
        'City':mensaje['City'],
        'Location': mensaje['Location'],
        'Latitud' : mensaje['Latitud'],
        'Longitud' :  mensaje['Longitud'],
        'Source':mensaje['Source'],
        'Brand' : mensaje['Brand'],
        'Category': mensaje['Category'],
        'Created_at': mensaje['Created_at'],
        'Date' : mensaje['Created_at'].split()[0],
        'Hour' : int(mensaje['Created_at'].split()[1].split(':')[0])
           }

In [ ]:
mensaje

In [ ]:
int(mensaje['Created_at'].split()[1].split(':')[0])

In [ ]:
mensaje['Created_at'].split()[0] # date